my_strategy() with:
- n depth recursion
- greedy search

Typical losses / 1000:

- depth 0: 5.5%, 4.2% in 5 seconds
- depth 1: 10.0%, 11.1%, 12.1% in 45 seconds
- depth 2: 2.9%, 2.7% in 3.5 mins 
- depth 3: 3.2%, 3.0% in 27 mins

Typical losses / 100

- depth 4: 1, 0, 1, 4, 0 in c.15 mins / game (150 mins / 1000)
- on average 1.2% losses

NB with depth 4 I had to disable the timeout as moves are starting to take > 1s

HYPOTHESIS: looks like results worsen for ODD depths ??

In [21]:
# CONNECT 4 - v3 - GREEDY RECURSION

import numpy as np

# CONSTANTS

# Minimax search depth
DEPTH = 1
# Game definition
ROWS = 6
COLUMNS = 7
CONNECT_N = 4
# Player definition - values ascribed later
AI_PLAYER = None
HUMAN_PLAYER = None
# Enable debug output
DEBUG_OUTPUT = False
# Board and player colours
COLOUR_EMPTY = "\u2022 "
COLOUR_RED = "\U0001F534"
COLOUR_YELLOW = "\U0001F7E1"
COLOUR_BLACK = "\u26AB" # black just fills the space for testing
COLOUR_COUNTER=[COLOUR_EMPTY, COLOUR_RED, COLOUR_YELLOW, COLOUR_BLACK]
COLOUR_TEXT = ["Blank", "Red", "Yellow", "Black"]

def my_strategy(board, player_num):
    # To interface into tester
    # converts list of lists [[int]*6]*7 into np array
    global AI_PLAYER, HUMAN_PLAYER
    AI_PLAYER = player_num
    HUMAN_PLAYER = AI_PLAYER ^ 3
    return get_ai_move(np.array(board))

def print_debug(str):
    # Print debug output if enabled
    if DEBUG_OUTPUT:
        print(str)

def flip_coin():
    # Returns True 50% of the time
    return np.random.randint(2) == 1
    
def get_valid_moves(board):
    # Returns a list of valid moves for the current board state
    valid_moves = []
    for col in range(COLUMNS):
        if board[col,ROWS-1] == 0:
            valid_moves.append(col)
    return valid_moves

def is_terminal_node(board):
    # Check for Terminal Node (Win, Draw, or Continue)
    return check_winner(board, AI_PLAYER) or check_winner(board, HUMAN_PLAYER) or check_draw(board)

def check_winner(board, player):
    # Check horizontal, vertical, and diagonal lines for a win
    for col in range(COLUMNS):
        for row in range(ROWS):
            # If cell is empty, no need to check this cell or cells above
            if board[col, row] == 0:
                break
            # If cell is not the player, skip but keep checking cells above
            if board[col, row] != player:
                continue
            # If room above... check vertical
            if (row + CONNECT_N <= ROWS):
                if all (board[col, row + i] == player for i in range(1, CONNECT_N)):
                    return True
            # If room to the right...
            if (col + CONNECT_N <= COLUMNS):  # if room to the right
                # ...check horizontal-right
                if all (board[col + i, row] == player for i in range(1, CONNECT_N)):
                    return True
                # ...check diagonal up-right if room above
                if (row + CONNECT_N <= ROWS):
                    if all (board[col + i, row + i] == player for i in range(1, CONNECT_N)):
                        return True
                # ...check diagonal down-right if room below
                if (row - CONNECT_N >= -1):
                    if all (board[col + i, row - i] == player for i in range(1, CONNECT_N)):
                        return True
    return False

def check_draw(board):
    return all (board[column, ROWS-1] !=0 for column in range(COLUMNS))

def evaluate_board(board, col, row):

    def eval_get_counts(board, player_num, column, row, column_step, row_step):
    # Returns the number of adjacent, separate, and blank cells in a line

        def is_in_range(x, max):
            return x >= 0 and x < max

        count_same_adj = 0
        count_same_sep = 0
        count_blank = 0

        # evaluate in both directions along the line dir -1 and 1
        for eval_dir in range(-1,2,2):
            check_column = column + column_step * eval_dir
            check_row = row + row_step * eval_dir
            check_adjacent = True
            # explore up to CONNECT_N-1 spaces either side
            for depth in range(CONNECT_N - 1):
                # stop if off the board
                if not (is_in_range(check_column,7) and is_in_range(check_row,6)):
                    break
                # stop if cell is opponent's
                if board[check_column,check_row] == player_num ^ 3:
                    break
                # if the cell is a blank, count it but set adjacent to false
                if board[check_column,check_row] == 0:
                    count_blank += 1
                    check_adjacent = False
                # if the cell is the same colour, count adjacent or separate
                if board[check_column,check_row] == player_num:
                    if check_adjacent:
                        count_same_adj += 1
                    else:
                        count_same_sep += 1
                check_column += column_step * eval_dir
                check_row += row_step * eval_dir
        return count_same_adj, count_same_sep, count_blank

    def eval_line(board, column, row, column_step, row_step):
    # Returns a line score based on the number of adjacent, separate, and blank cells
    # Score is from the perspective of last player i.e. player_num in [col,row]

        # evaluation weightings
        EVAL_SAME_ADJACENT = 2 # exponential
        EVAL_SAME_SEPARATE = 1 # linear
        EVAL_BLANK = 1 # linear

        eval_line_score = 0
        player_num = board[column,row]

        # Calculate offensive score
        count_same_adj, count_same_sep, count_blank = eval_get_counts(board, player_num, column, row, column_step, row_step)
        
        # Only score a line if there is enough space to win
        # NB function might return eval_line_score=0 (better than losing move)
        if (count_same_adj + count_same_sep + count_blank) >= CONNECT_N - 1:
            if count_same_adj > 0:
                eval_line_score += EVAL_SAME_ADJACENT ** count_same_adj
            eval_line_score += count_same_sep * EVAL_SAME_SEPARATE
            eval_line_score += count_blank * EVAL_BLANK
            print_debug(f"offense: count_same_adj: {count_same_adj}, count_same_sep: {count_same_sep}, count_blank: {count_blank}, eval score: {eval_line_score}")

        # Add a defensive score
        count_same_adj, count_same_sep, count_blank = eval_get_counts(board, player_num ^ 3, column, row, column_step, row_step)
        
        # Only score a line if there is enough space for opponent to win
        if (count_same_adj + count_same_sep + count_blank) >= CONNECT_N - 1:
            if count_same_adj > 0:
                eval_line_score += EVAL_SAME_ADJACENT ** count_same_adj
            eval_line_score += count_same_sep * EVAL_SAME_SEPARATE
            eval_line_score += count_blank * EVAL_BLANK
            print_debug(f"defense: count_same_adj: {count_same_adj}, count_same_sep: {count_same_sep}, count_blank: {count_blank}, eval score: {eval_line_score}")

        return eval_line_score

    def eval_move(board, column, row):
    # Returns a score for a move based on the evaluation of each line

        eval_move_score = 0
        eval_steps = np.array([[1, 0], [1, 1], [0, 1], [1, -1]])
        # Evaluate each line in turn
        for i in range(4):
            print_debug(f"evaluating line {i}")
            eval_move_score += eval_line(board, column, row, eval_steps[i,0], eval_steps[i,1])
        print_debug(f"column {column} score: {eval_move_score}")
        return eval_move_score

    # Check for a win, loss, or draw, otherwise return a heuristic evaluation
    if check_winner(board, AI_PLAYER):
        return np.inf  # AI wins
    elif check_winner(board, HUMAN_PLAYER):
        return -np.inf  # Human wins
    elif check_draw(board):
        return 0  # Draw
    else:
        # Heuristic evaluation for intermediate states (foccsing on the move that led to this board state)
        return eval_move(board, col, row)

def minimax(board, col, row, depth, maximizing_player):
    # Recursive Analysis Function
    # This is a generic gaming function that can be used for any two-player game
    # Each level represents a player's turn, alternating between maximizing and minimizing players

    # Returns a score for the board at depth 0 or at terminal node along each path (whatever comes first)
    # The maximizing player tries to get the highest score, while the minimizing player tries to get the lowest score
    # The function returns the best score for the maximizing player at the top level

    # This is a 'greedy' search algorithm that iterates down every possible path
    # A faster option is to add alpha-beta pruning to cut off paths that are worse than the current best path
    # col row (for the move that led to this board state) are passed to the board evaluation function

    if depth == 0 or is_terminal_node(board):
        return evaluate_board(board, col, row)
    
    valid_moves = get_valid_moves(board)
    if maximizing_player:
        value = -np.inf
        for col in valid_moves:
            new_board = board.copy()
            row = make_move(new_board, col, AI_PLAYER)  # AI is the maximizing player
            value = max(value, minimax(new_board, col, row, depth-1, False))
        # Returns the most positive score (a high score is good for AI, bad for Human)
        # i.e. 'how good it could be for AI' if AI makes the best move
        if depth == DEPTH:
            print_debug(f"AI value (best case): {value}")
        return value
    else:
        value = np.inf
        for col in valid_moves:
            new_board = board.copy()
            row = make_move(new_board, col, HUMAN_PLAYER)  # Human is the minimizing player
            value = min(value, minimax(new_board, col, row, depth-1, True))
        # Returns most negative score (a high score is good for AI, bad for Human)
        # i.e. 'how bad it could be for AI' if human makes the best move
        if depth == DEPTH:
            print_debug(f"Human value (worst case): {value}")
        return value

def make_move(board, col, player):
    # Updates the board col with the player's move amd return the row
    row = next(r for r in range(ROWS) if board[col, r] == 0)
    board[col, row] = player
    return row

def get_ai_move(board):
    # Returns the best move for the AI player
    valid_moves = get_valid_moves(board)
    best_score = -np.inf
    best_col = valid_moves[0]  # Default - just in case the only moves are losing
    for col in valid_moves:
        new_board = board.copy()
        row = make_move(new_board, col, AI_PLAYER)
        # This initiates the minimax recursion
        # Because the minimax recursion starts with maximizing_player = False, it will return the
        # worst-case board evaluation of all possible paths at depth 0 (or terminal nodes),
        # assuming the human (and then AI) both play perfectly
        score = minimax(new_board, col, row, DEPTH, False)
        # So choose the 'least worse' option for AI
        if score > best_score or (score == best_score and flip_coin()):
            best_score = score
            best_col = col
    return best_col

In [22]:
# When you're ready to run your strategy run the top cell, then this cell
# You can do this as often as you like as you improve your strategy
from assessment.assessor import assess

assess(my_strategy, 1000)

Assessing student strategy...
                                                                                                    
Results
Wins: 879
Draws: 0
Losses: 121
Forfeits: 0
Mark: 75.94%
